In [1]:
import os, glob
from utils import (prepare_sequences, 
                   add_temporal_features, 
                   cv_leave_one_out_split,
                   create_feature_combinations,
                   get_temporal_feature_names,
                   create_optuna_objective,
                   get_date_stats,
                   custom_error_matrix,
                   custom_f1_score,
                   calculate_mape_and_offset
                   )

os.chdir('/mnt/d/OneDrive - Univerzita Komenskeho v Bratislave/03.PhD/Projects/00_GMI')

import pandas as pd
import geopandas as gpd
from tqdm import tqdm

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score, roc_auc_score
import optuna
import sys
sys.path.append('/mnt/d/OneDrive - Univerzita Komenskeho v Bratislave/03.PhD/Projects/2.Chapter/code')
from management import remove_close_mowing

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [10]:
# Load management events reference data
points = gpd.read_file('DATA/KOSENIE/GMI_Long.gpkg')

# Get paths to satellite time-series data
s1_paths = glob.glob('DATA/GMI_TS/S1/*csv')
s2_paths = glob.glob('DATA/GMI_TS/S2/*csv')
coh_paths = glob.glob('DATA/GMI_TS/COH/*csv')

# Define columns to load
input_id_cols = ['PLOTID', 'Date']

s1_cols = [
    'VH', 'VH_asm', 'VH_corr', 'VH_ent',
    'VV', 'VV_asm', 'VV_corr', 'VV_ent',
    'rvi', 'rvi_asm', 'rvi_corr', 'rvi_ent'
] + input_id_cols

s2_cols = [
    'blue', 'green', 'red', 'nir', 're1', 're2', 're3',
    'swir1', 'swir2', 'ndvi', 'spatial_max',
    'ndvi_texture_sd_3', 'ndvi_texture_sd_5'
] + input_id_cols

coh_cols = ['b1', 'b2'] + input_id_cols

# Get list of plot IDs
PLOTIDS = sorted(list(points['PLOTID'].unique()))

print(f"Total plots: {len(PLOTIDS)}")
print(f"Total management events: {len(points)}")
print(f"\nManagement event types:")
print(points['type'].value_counts())

Total plots: 108
Total management events: 254

Management event types:
type
k    217
n     37
Name: count, dtype: int64


In [3]:
# Create grid of feature combinations
df_grid = create_feature_combinations()
print(f"Total feature combinations to test: {len(df_grid)}")

Total feature combinations to test: 70


In [8]:
from sklearn.metrics import  make_scorer

def objective(trial):
    # --- Definícia hyperparametrov ---
    n_estimators = trial.suggest_int("n_estimators", 200, 600, step=100)
    max_features = trial.suggest_categorical("max_features", ['sqrt', 'log2'])
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 2, 10, step=2)
    max_leaf_nodes = trial.suggest_categorical("max_leaf_nodes", [None, 50, 100, 200, 500])
    bootstrap = trial.suggest_categorical("bootstrap", [True, False])
    class_weight = trial.suggest_categorical("class_weight", ["balanced", "balanced_subsample"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_features=max_features,
        min_samples_leaf=min_samples_leaf,
        max_leaf_nodes=max_leaf_nodes,
        bootstrap=bootstrap,
        class_weight=class_weight,
        random_state=42,
        n_jobs=-1
    )

    # --- Cross-val F1 scoring ---
    scores = cross_val_score(model, X_train, y_train, cv=2, scoring=make_scorer(f1_score), n_jobs=-1)
    return scores.mean()

In [5]:
# Configuration
N_OPTUNA_TRIALS = 30
split_col = 'id'
id_cols = ['PLOTID', 'Date', 'year', 'mowing', 'id']
all_results = []
plotID_col = 'PLOTID'
optuna.logging.set_verbosity(optuna.logging.WARNING)

for grid_idx, config in tqdm(df_grid.iterrows(), total=len(df_grid), desc="Feature combinations"):
    experiment_name = config['input_name']
    input_variables = config['input_vars']
    use_s1 = config['use_s1']
    
    print(f"\n{'='*80}")
    print(f"Experiment {grid_idx + 1}/{len(df_grid)}: {experiment_name}")
    print(f"{'='*80}")
    
    # ========== 1. PREPARE DATA ==========
    df = prepare_sequences(
        points=points,
        s1_paths=s1_paths,
        s2_paths=s2_paths,
        coh_paths=coh_paths,
        s1_cols=s1_cols,
        s2_cols=s2_cols,
        coh_cols=coh_cols,
        use_s1=use_s1,
    )
    
    # Filter to relevant plots
    df = df[df['PLOTID'].isin(PLOTIDS)]
    
    # Filter to years with management events
    mask = df[['PLOTID', 'year']].apply(tuple, axis=1).isin(
        points[['PLOTID', 'year']].apply(tuple, axis=1)
    )
    df = df[mask]
    df = df.drop(columns='grazing')
    
    # ========== 2. ADD DERIVED FEATURES ==========
    if 'VV' in input_variables:
        df['rnd'] = (df['VV'] - df['VH']) / (df['VV'] + df['VH'])
    
    if 'ndvi' in input_variables:
        df['diff_max'] = df['ndvi'] - df['spatial_max']
        df['bsi'] = (
            (df["swir1"] + df["red"]) - (df["nir"] - df["blue"])
            ) / (
            (df["swir1"] + df["red"]) + (df["nir"] + df["blue"])
            )
    
    df = df[input_variables + id_cols]
    
    # ========== 3. ADD TEMPORAL FEATURES (ONLY PAST) ==========
    df = add_temporal_features(
        df=df,
        columns=input_variables,
        group_cols=['PLOTID', 'year'],
        date_col='Date',
        past_steps=1
    )
    
    temporal_features = get_temporal_feature_names(
        base_features=input_variables,
        past_steps=1,
    )
    
    df = df[id_cols + input_variables + temporal_features]
    df = df.dropna(subset=temporal_features)
    df['doy'] = df['Date'].dt.day_of_year
    
    print(f"  Data shape: {df.shape}")
    print(f"  Mowing events: {df['mowing'].sum()} / {len(df)} ({df['mowing'].mean()*100:.2f}%)")
    
    # ========== 4. LEAVE-ONE-REGION-OUT CROSS-VALIDATION ==========
    for test_region, df_train, df_test in cv_leave_one_out_split(df, split_col=split_col):
        
        print(f"\n  Testing on region: {test_region}")
        
        # Prepare train/test splits
        X_train = df_train[temporal_features].copy()
        y_train = df_train['mowing'].copy()
        groups_train = df_train[split_col].values  # Extract groups for spatial CV
        
        X_test = df_test[temporal_features].copy()
        y_test = df_test['mowing'].copy()
        
        # ========== 5. HYPERPARAMETER OPTIMIZATION ==========
        print(f"  Optimizing hyperparameters using spatial CV on training regions...")
        
        # Create objective function
        objective_function = create_optuna_objective(
            X_train=X_train,
            y_train=y_train,
            groups=groups_train,  # Pass spatial groups
            cv_folds=2  # With 2 training regions, use 2-fold GroupKFold
        )
        
        # Create and run Optuna study
        study = optuna.create_study(
            direction="maximize",
            study_name=f"rf_{experiment_name}_{test_region}",
            sampler=optuna.samplers.TPESampler(seed=34)
        )
        
        study.optimize(
            objective_function,
            n_trials=N_OPTUNA_TRIALS,
            n_jobs=-1,
            show_progress_bar=False
        )
        
        best_params = study.best_trial.params
        best_cv_f1 = study.best_trial.value
        
        print(f"    Best CV F1 (training regions): {best_cv_f1:.4f}")
        print(f"    Best params: {best_params}")
        model = RandomForestClassifier(
            n_estimators=best_params['n_estimators'],
            max_features=best_params['max_features'],
            min_samples_leaf=best_params['min_samples_leaf'],
            max_leaf_nodes=best_params['max_leaf_nodes'],
            bootstrap=best_params['bootstrap'],
            max_samples=best_params.get('max_samples', None),
            random_state=42,
            n_jobs=-1
        )
        
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.predict(X_test)
        y_pred_bin = (y_pred >= 0.5).astype(int)
        df_test = df_test.copy()
        df_test['pred_mowing'] = y_pred_bin

        if plotID_col not in df_test.columns:
            df_test[plotID_col] = df_test['PLOTID']  # fallback: use PLOTID

        if 'doy' not in df_test.columns:
            df_test['doy'] = df_test['Date'].dt.day_of_year

        # --- Remove Close Mowing (15 days)
        df_test = df_test.groupby(['PLOTID', 'year'], group_keys=False).apply(remove_close_mowing, 'pred_mowing')

        # --- Error matrix and validation metrics ---
        metrics = custom_error_matrix(
            df_test,
            df_test['pred_mowing'],
            'mowing'
        )

        date_metrics = get_date_stats(df_test, points, df_test['pred_mowing'], 'PLOTID')

        res = (
            df_test.groupby(['PLOTID', 'year'])
            .agg(preds=('pred_mowing', 'sum'), ref=('mowing', 'sum'))
            .reset_index()[['preds', 'ref']]
        )

        mape = calculate_mape_and_offset(res['preds'], res['ref'])
        auc = roc_auc_score(df_test['mowing'], df_test['pred_mowing'])

        # --- Save results ---
        all_results.append({
            'grid_idx': grid_idx,
            'input_name': config['input_name'],
            'test_region': test_region, #region if 'region' in locals() else None,
            'f1': metrics['F1'],
            'auc': auc,
            'precision': metrics['Precision'],
            'recall': metrics['Recall'],
            'mape': mape[0],
            'mape_offset': mape[1],
            'tp': metrics['TP'],
            'fp': metrics['FP'],
            'fn': metrics['FN'],
            'tn': metrics['TN'],
            'rmse': date_metrics['RMSE'],
            'mae': date_metrics['MAE'],
            'r2': date_metrics['R²'],
            'max_error': date_metrics['Max Error'],
            'mead_deviance': date_metrics['Mean Deviance'],
        })



Feature combinations:   0%|          | 0/70 [00:00<?, ?it/s]


Experiment 1/70: coh
  Data shape: (9311, 10)
  Mowing events: 208 / 9311 (2.23%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: ['LR', 'NB', 'NS']
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = LR
  Train regions: ['NB', 'NS']
  Train: 5862 samples from 2 regions
  Test:  3449 samples from 1 region

  Testing on region: LR
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0100
    Best params: {'n_estimators': 600, 'max_features': 'log2', 'min_samples_leaf': 1, 'max_leaf_nodes': 500, 'bootstrap': False}

Iteration: Test region = NB
  Train regions: ['LR', 'NS']
  Train: 6726 samples from 2 regions
  Test:  2585 samples from 1 region

  Testing on region: NB
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0100
    Best params: {'n_estimators': 600, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'max_leaf_nodes': 50

Feature combinations:   1%|▏         | 1/70 [01:55<2:13:01, 115.68s/it]


Experiment 2/70: optical
  Data shape: (9216, 24)
  Mowing events: 208 / 9216 (2.26%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: ['LR', 'NB', 'NS']
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = LR
  Train regions: ['NB', 'NS']
  Train: 5193 samples from 2 regions
  Test:  4023 samples from 1 region

  Testing on region: LR
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.7607
    Best params: {'n_estimators': 400, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'max_leaf_nodes': 100, 'bootstrap': True, 'max_samples': 0.7}

Iteration: Test region = NB
  Train regions: ['LR', 'NS']
  Train: 7591 samples from 2 regions
  Test:  1625 samples from 1 region

  Testing on region: NB
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.7462
    Best params: {'n_estimators': 200, 'max_features': 'sqrt', 'min_samples_leaf': 

Feature combinations:   3%|▎         | 2/70 [03:53<2:12:44, 117.12s/it]

...

Feature combinations:  97%|█████████▋| 68/70 [2:11:19<03:47, 113.97s/it]


Experiment 69/70: vi_texture+radar+r_texture+coh
  Data shape: (9311, 40)
  Mowing events: 208 / 9311 (2.23%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: ['LR', 'NB', 'NS']
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = LR
  Train regions: ['NB', 'NS']
  Train: 5862 samples from 2 regions
  Test:  3449 samples from 1 region

  Testing on region: LR
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0424
    Best params: {'n_estimators': 600, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'max_leaf_nodes': 50, 'bootstrap': False}

Iteration: Test region = NB
  Train regions: ['LR', 'NS']
  Train: 6726 samples from 2 regions
  Test:  2585 samples from 1 region

  Testing on region: NB
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0000
    Best params: {'n_estimators': 200, 'max_features': 'sqrt', 'min_samples_lea

Feature combinations:  99%|█████████▊| 69/70 [2:13:16<01:54, 114.93s/it]


Experiment 70/70: vi_texture+s1_grid
  Data shape: (9311, 10)
  Mowing events: 208 / 9311 (2.23%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: ['LR', 'NB', 'NS']
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = LR
  Train regions: ['NB', 'NS']
  Train: 5862 samples from 2 regions
  Test:  3449 samples from 1 region

  Testing on region: LR
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.1358
    Best params: {'n_estimators': 500, 'max_features': 'log2', 'min_samples_leaf': 3, 'max_leaf_nodes': 100, 'bootstrap': False}

Iteration: Test region = NB
  Train regions: ['LR', 'NS']
  Train: 6726 samples from 2 regions
  Test:  2585 samples from 1 region

  Testing on region: NB
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0880
    Best params: {'n_estimators': 600, 'max_features': 'log2', 'min_samples_leaf': 2, 'max

Feature combinations: 100%|██████████| 70/70 [2:15:21<00:00, 116.02s/it]


In [6]:
results_df = pd.DataFrame(all_results)
summary = results_df.groupby(['grid_idx', 'input_name', 'test_region']).agg(
    mean_f1=('f1', 'mean'),
    mean_auc=('auc', 'mean'),
    mean_precision=('precision', 'mean'),
    mean_recall=('recall', 'mean'),
    mean_mape=('mape', 'mean'),
    mean_mape_offset =('mape_offset', 'mean'),
    tp_evt = ('tp', 'mean'),
    fp_evt = ('fp', 'mean'),
    fn_evt = ('fn', 'mean'),
    tn_evt = ('tn', 'mean'),
    rmse = ('rmse', 'mean'),
    mae = ('mae', 'mean'),
    r2 = ('r2', 'mean'),
    max_error = ('max_error', 'mean'),
    mead_deviance = ('mead_deviance', 'mean'),
).reset_index().sort_values('mean_f1', ascending=False)
results_df.to_csv('results.csv')

In [8]:
# Configuration
N_OPTUNA_TRIALS = 30
split_col = 'year'
id_cols = ['PLOTID', 'Date', 'year', 'mowing', 'id']
all_results = []
plotID_col = 'PLOTID'
optuna.logging.set_verbosity(optuna.logging.WARNING)

for grid_idx, config in tqdm(df_grid.iterrows(), total=len(df_grid), desc="Feature combinations"):
    experiment_name = config['input_name']
    input_variables = config['input_vars']
    use_s1 = config['use_s1']
    
    print(f"\n{'='*80}")
    print(f"Experiment {grid_idx + 1}/{len(df_grid)}: {experiment_name}")
    print(f"{'='*80}")
    
    # ========== 1. PREPARE DATA ==========
    df = prepare_sequences(
        points=points,
        s1_paths=s1_paths,
        s2_paths=s2_paths,
        coh_paths=coh_paths,
        s1_cols=s1_cols,
        s2_cols=s2_cols,
        coh_cols=coh_cols,
        use_s1=use_s1,
    )
    
    # Filter to relevant plots
    df = df[df['PLOTID'].isin(PLOTIDS)]
    
    # Filter to years with management events
    mask = df[['PLOTID', 'year']].apply(tuple, axis=1).isin(
        points[['PLOTID', 'year']].apply(tuple, axis=1)
    )
    df = df[mask]
    df = df.drop(columns='grazing')
    
    # ========== 2. ADD DERIVED FEATURES ==========
    if 'VV' in input_variables:
        df['rnd'] = (df['VV'] - df['VH']) / (df['VV'] + df['VH'])
    
    if 'ndvi' in input_variables:
        df['diff_max'] = df['ndvi'] - df['spatial_max']
        df['bsi'] = (
            (df["swir1"] + df["red"]) - (df["nir"] - df["blue"])
            ) / (
            (df["swir1"] + df["red"]) + (df["nir"] + df["blue"])
            )
    
    df = df[input_variables + id_cols]
    
    # ========== 3. ADD TEMPORAL FEATURES (ONLY PAST) ==========
    df = add_temporal_features(
        df=df,
        columns=input_variables,
        group_cols=['PLOTID', 'year'],
        date_col='Date',
        past_steps=1
    )
    
    temporal_features = get_temporal_feature_names(
        base_features=input_variables,
        past_steps=1,
    )
    
    df = df[id_cols + input_variables + temporal_features]
    df = df.dropna(subset=temporal_features)
    df['doy'] = df['Date'].dt.day_of_year
    
    print(f"  Data shape: {df.shape}")
    print(f"  Mowing events: {df['mowing'].sum()} / {len(df)} ({df['mowing'].mean()*100:.2f}%)")
    
    # ========== 4. LEAVE-ONE-REGION-OUT CROSS-VALIDATION ==========
    for test_region, df_train, df_test in cv_leave_one_out_split(df, split_col=split_col):
        
        print(f"\n  Testing on region: {test_region}")
        
        # Prepare train/test splits
        X_train = df_train[temporal_features].copy()
        y_train = df_train['mowing'].copy()
        groups_train = df_train[split_col].values  # Extract groups for spatial CV
        
        X_test = df_test[temporal_features].copy()
        y_test = df_test['mowing'].copy()
        
        # ========== 5. HYPERPARAMETER OPTIMIZATION ==========
        print(f"  Optimizing hyperparameters using spatial CV on training regions...")
        
        # Create objective function
        objective_function = create_optuna_objective(
            X_train=X_train,
            y_train=y_train,
            groups=groups_train,  # Pass spatial groups
            cv_folds=2  # With 2 training regions, use 2-fold GroupKFold
        )
        
        # Create and run Optuna study
        study = optuna.create_study(
            direction="maximize",
            study_name=f"rf_{experiment_name}_{test_region}",
            sampler=optuna.samplers.TPESampler(seed=34)
        )
        
        study.optimize(
            objective_function,
            n_trials=N_OPTUNA_TRIALS,
            n_jobs=-1,
            show_progress_bar=False
        )
        
        best_params = study.best_trial.params
        best_cv_f1 = study.best_trial.value
        
        print(f"    Best CV F1 (training regions): {best_cv_f1:.4f}")
        print(f"    Best params: {best_params}")
        model = RandomForestClassifier(
            n_estimators=best_params['n_estimators'],
            max_features=best_params['max_features'],
            min_samples_leaf=best_params['min_samples_leaf'],
            max_leaf_nodes=best_params['max_leaf_nodes'],
            bootstrap=best_params['bootstrap'],
            max_samples=best_params.get('max_samples', None),
            random_state=42,
            n_jobs=-1
        )
        
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.predict(X_test)
        y_pred_bin = (y_pred >= 0.5).astype(int)
        df_test = df_test.copy()
        df_test['pred_mowing'] = y_pred_bin

        if plotID_col not in df_test.columns:
            df_test[plotID_col] = df_test['PLOTID']  # fallback: use PLOTID

        if 'doy' not in df_test.columns:
            df_test['doy'] = df_test['Date'].dt.day_of_year

        # --- Remove Close Mowing (15 days)
        df_test = df_test.groupby(['PLOTID', 'year'], group_keys=False).apply(remove_close_mowing, 'pred_mowing')

        # --- Error matrix and validation metrics ---
        metrics = custom_error_matrix(
            df_test,
            df_test['pred_mowing'],
            'mowing'
        )

        date_metrics = get_date_stats(df_test, points, df_test['pred_mowing'], 'PLOTID')

        res = (
            df_test.groupby(['PLOTID', 'year'])
            .agg(preds=('pred_mowing', 'sum'), ref=('mowing', 'sum'))
            .reset_index()[['preds', 'ref']]
        )

        mape = calculate_mape_and_offset(res['preds'], res['ref'])
        auc = roc_auc_score(df_test['mowing'], df_test['pred_mowing'])

        # --- Save results ---
        all_results.append({
            'grid_idx': grid_idx,
            'input_name': config['input_name'],
            'test_region': test_region, #region if 'region' in locals() else None,
            'f1': metrics['F1'],
            'auc': auc,
            'precision': metrics['Precision'],
            'recall': metrics['Recall'],
            'mape': mape[0],
            'mape_offset': mape[1],
            'tp': metrics['TP'],
            'fp': metrics['FP'],
            'fn': metrics['FN'],
            'tn': metrics['TN'],
            'rmse': date_metrics['RMSE'],
            'mae': date_metrics['MAE'],
            'r2': date_metrics['R²'],
            'max_error': date_metrics['Max Error'],
            'mead_deviance': date_metrics['Mean Deviance'],
        })



Feature combinations:   0%|          | 0/70 [00:00<?, ?it/s]


Experiment 1/70: coh
  Data shape: (9311, 10)
  Mowing events: 208 / 9311 (2.23%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: [2021, 2022, 2023]
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = 2021
  Train regions: [2022, 2023]
  Train: 4720 samples from 2 regions
  Test:  4591 samples from 1 region

  Testing on region: 2021
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0000
    Best params: {'n_estimators': 200, 'max_features': 'log2', 'min_samples_leaf': 10, 'max_leaf_nodes': 500, 'bootstrap': False}

Iteration: Test region = 2022
  Train regions: [2021, 2023]
  Train: 6933 samples from 2 regions
  Test:  2378 samples from 1 region

  Testing on region: 2022
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0000
    Best params: {'n_estimators': 200, 'max_features': 'log2', 'min_samples_leaf': 8, 'max_leaf_n

Feature combinations:   1%|▏         | 1/70 [01:55<2:12:53, 115.56s/it]


Experiment 2/70: optical
  Data shape: (9216, 24)
  Mowing events: 208 / 9216 (2.26%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: [2021, 2022, 2023]
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = 2021
  Train regions: [2022, 2023]
  Train: 6154 samples from 2 regions
  Test:  3062 samples from 1 region

  Testing on region: 2021
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.7711
    Best params: {'n_estimators': 300, 'max_features': 'log2', 'min_samples_leaf': 2, 'max_leaf_nodes': None, 'bootstrap': False}

Iteration: Test region = 2022
  Train regions: [2021, 2023]
  Train: 6006 samples from 2 regions
  Test:  3210 samples from 1 region

  Testing on region: 2022
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.7795
    Best params: {'n_estimators': 500, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'max_le

Feature combinations:   3%|▎         | 2/70 [03:52<2:11:59, 116.46s/it]

...

Feature combinations:  97%|█████████▋| 68/70 [2:13:41<03:49, 114.73s/it]


Experiment 69/70: vi_texture+radar+r_texture+coh
  Data shape: (9311, 40)
  Mowing events: 208 / 9311 (2.23%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: [2021, 2022, 2023]
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = 2021
  Train regions: [2022, 2023]
  Train: 4720 samples from 2 regions
  Test:  4591 samples from 1 region

  Testing on region: 2021
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0233
    Best params: {'n_estimators': 300, 'max_features': 'log2', 'min_samples_leaf': 1, 'max_leaf_nodes': 500, 'bootstrap': False}

Iteration: Test region = 2022
  Train regions: [2021, 2023]
  Train: 6933 samples from 2 regions
  Test:  2378 samples from 1 region

  Testing on region: 2022
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.0392
    Best params: {'n_estimators': 200, 'max_features': 'sqrt', 'min_sa

Feature combinations:  99%|█████████▊| 69/70 [2:15:35<01:54, 114.48s/it]


Experiment 70/70: vi_texture+s1_grid
  Data shape: (9311, 10)
  Mowing events: 208 / 9311 (2.23%)
Leave-One-Region-Out Cross-Validation:
  Total regions: 3
  Regions: [2021, 2022, 2023]
  Will perform 3 iterations (1 region held out each time)

Iteration: Test region = 2021
  Train regions: [2022, 2023]
  Train: 4720 samples from 2 regions
  Test:  4591 samples from 1 region

  Testing on region: 2021
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.1003
    Best params: {'n_estimators': 600, 'max_features': 'log2', 'min_samples_leaf': 1, 'max_leaf_nodes': None, 'bootstrap': False}

Iteration: Test region = 2022
  Train regions: [2021, 2023]
  Train: 6933 samples from 2 regions
  Test:  2378 samples from 1 region

  Testing on region: 2022
  Optimizing hyperparameters using spatial CV on training regions...
    Best CV F1 (training regions): 0.1429
    Best params: {'n_estimators': 300, 'max_features': 'sqrt', 'min_samples_leaf'

Feature combinations: 100%|██████████| 70/70 [2:17:25<00:00, 117.79s/it]


In [9]:
results_df = pd.DataFrame(all_results)
summary = results_df.groupby(['grid_idx', 'input_name', 'test_region']).agg(
    mean_f1=('f1', 'mean'),
    mean_auc=('auc', 'mean'),
    mean_precision=('precision', 'mean'),
    mean_recall=('recall', 'mean'),
    mean_mape=('mape', 'mean'),
    mean_mape_offset =('mape_offset', 'mean'),
    tp_evt = ('tp', 'mean'),
    fp_evt = ('fp', 'mean'),
    fn_evt = ('fn', 'mean'),
    tn_evt = ('tn', 'mean'),
    rmse = ('rmse', 'mean'),
    mae = ('mae', 'mean'),
    r2 = ('r2', 'mean'),
    max_error = ('max_error', 'mean'),
    mead_deviance = ('mead_deviance', 'mean'),
).reset_index().sort_values('mean_f1', ascending=False)
results_df.to_csv('results_year.csv')

In [10]:
summary

,grid_idx,input_name,test_region,mean_f1,mean_auc,mean_precision,mean_recall,mean_mape,mean_mape_offset,tp_evt,fp_evt,fn_evt,tn_evt,rmse,mae,r2,max_error,mead_deviance
30,10,optical+vi,2021,0.903226,0.871966,0.982456,0.835821,0.220779,0.233766,56.0,1.0,11.0,2994.0,2.854986,0.679245,0.983428,12.0,0.679245
57,19,optical+vi+vi_texture,2021,0.901639,0.872300,1.000000,0.820896,0.188312,0.207792,55.0,0.0,12.0,2995.0,13.273608,2.339623,0.618569,95.0,2.339623
31,10,optical+vi,2022,0.890625,0.863171,0.982759,0.814286,0.195122,0.195122,57.0,1.0,13.0,3139.0,4.478121,0.982143,0.913385,30.0,-0.267857
3,1,optical,2021,0.888889,0.871632,0.949153,0.835821,0.253247,0.285714,56.0,3.0,11.0,2992.0,28.203587,7.096154,-0.711881,145.0,-6.173077
58,19,optical+vi+vi_texture,2022,0.875000,0.863171,0.965517,0.800000,0.219512,0.219512,56.0,2.0,14.0,3138.0,4.164787,0.763636,0.915228,30.0,-0.509091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,41,radar+r_texture,2021,0.000000,0.500000,0.000000,0.000000,0.831169,0.870130,0.0,0.0,67.0,4524.0,NaN,NaN,NaN,NaN,NaN
124,41,radar+r_texture,2022,0.000000,0.500000,0.000000,0.000000,0.853659,0.853659,0.0,0.0,70.0,2308.0,NaN,NaN,NaN,NaN,NaN
125,41,radar+r_texture,2023,0.000000,0.500000,0.000000,0.000000,0.848101,0.898734,0.0,0.0,71.0,2271.0,NaN,NaN,NaN,NaN,NaN
126,42,radar+r_texture+coh,2021,0.000000,0.500000,0.000000,0.000000,0.831169,0.870130,0.0,0.0,67.0,4524.0,NaN,NaN,NaN,NaN,NaN
